**CMB emulator training tutorial-compressed way**\
This tutorial goes through how to train an emulator for CMB power spectrum. This tutorial, different from the other tutorial for training, compress the codes of defining models and training process in a package called **emulator**. This one is intended to be more user-friendly, while the other one is intended more for people to understand the process of developing the code and can modify codes according to their own need. We use CMB TT power spectrum as the example here, and same strategies will apply to EE and TE. 

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import scipy
import camb
import scipy.linalg
from camb import model, initialpower
from camb.dark_energy import DarkEnergyPPF, DarkEnergyFluid
import emulator
from emulator import Supact, Affine, Better_Attention, Better_Transformer, ResBlock, ResMLP, TRF, train

In [2]:
##### SET UP CMB POWER SPECTRA RANGE #####

camb_ell_min          = 2
camb_ell_max          = 5000
camb_ell_range        = camb_ell_max - camb_ell_min

##### PICK DEVICE ON WHICH THE MODEL WILL BE TRAINED ON. WE RECOMMEND USING GPU FOR TRAINING #####
device                = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#CUDA for GPU
#CPU for CPU
#GPU is generally recommended for higher speed.


In [3]:
##### LOAD UP MEAN AND STD FOR INPUT AND OUTPUT #####
extrainfo=np.load("extra/extrainfo_plk_tt_first_T256.npy",allow_pickle=True)
X_mean=torch.Tensor(extrainfo.item()['X_mean'])#.to(device)
X_std=torch.Tensor(extrainfo.item()['X_std'])#.to(device)
Y_mean=torch.Tensor(extrainfo.item()['Y_mean']).to(device)
Y_std=torch.Tensor(extrainfo.item()['Y_std']).to(device)

##### LOAD UP COV MAT #####
covinv=np.load('/home/grads/data/yijie/mltrial/extra/cosvarinvTT.npy',allow_pickle=True)[:camb_ell_range,:camb_ell_range]
covinv=torch.Tensor(covinv).to(device) #This is inverse of the Covariance Matrix


#load in data
train_samples=np.load('parametersamples/cos_pkc_T256_firsttest.npy',allow_pickle=True)

validation_samples=np.load('parametersamples/cos_pkc_T128.npy',allow_pickle=True)[:10000]

train_data_vectors=np.load('datavectors/Planck256/cos_pkc_T256_firsttest_TT.npy',allow_pickle=True,mmap_mode='r+')

validation_data_vectors=np.load('datavectors/Planck128/cos_pkc_T128_TT_acc.npy',allow_pickle=True)[:10000,:4998]
train_samples=torch.Tensor(train_samples)
train_data_vectors=torch.Tensor(train_data_vectors)
validation_samples=torch.Tensor(validation_samples)
validation_data_vectors=torch.Tensor(validation_data_vectors)
#specifying input and output dimension of our model
input_size=len(train_samples[0])
out_size=len(train_data_vectors[0])



#normalizing samples and to mean 0, std 1

X_train=(train_samples-X_mean)/X_std
X_train[:,6:]=0 # we didn't vary the last 3 parameters: mnu, w, and wa in this test, so setting them to 0 automatically after normalization

X_validation=(validation_samples-X_mean)/X_std
X_validation[:,6:]=0 # we didn't vary the last 3 parameters: mnu, w, and wa in this test, so setting them to 0 automatically after normalization

X_train=X_train.to(torch.float32)
X_validation=X_validation.to(torch.float32)

X_mean=X_mean.to(device)
X_std=X_std.to(device)

#load the data to batches. Do not send those to device yet to save space
batch_size = 512

trainset    = TensorDataset(X_train, train_data_vectors)
validset    = TensorDataset(X_validation,validation_data_vectors)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=1)
validloader = DataLoader(validset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=1)


Here we start defining Machine learning Modules.

In [4]:
intdim = 4    # internal dimension of the ResMLP blocks
int_trf = 5120# internal dimension of the Transformer block
nc=32         # number of channels we pick

#Set up the model and optimizer
model = TRF(input_dim=input_size,output_dim=out_size,int_dim=intdim, int_trf=int_trf,N_channels=nc)
model = nn.DataParallel(model)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(),weight_decay=0)


In [5]:
# Setting up the learning rate scheduler
reduce_lr = True#reducing learning rate on plateau
if reduce_lr==True:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=15)

**TRAINING PROCESS**\
Here we use the loss function of $L=\sqrt{1+2\chi^2}$. The users should test out
their own best Loss functions and modify the code correspondingly.


In [6]:


n_epoch = 1 #just for demo, in reality you need around 500 to 700 or more epochs
PATH = "./trainedemu5000plktrf/chiTTAstautestc"+str(nc)
train(model, scheduler, optimizer, trainloader, validloader, n_epoch, covinv, device,X_mean, X_std, Y_mean, Y_std, PATH)


Reduce LR on plateu:  True
epoch 0, loss=16025.001953125, validation loss=5010.69677734375, lr=0.001, wd=0)
Saved to./trainedemu5000plktrf/chiTTAstautestc32.pt
